In [8]:
# Importing the required libraries.
from pymongo import MongoClient
import re

# Creating the connection to the gem2 server.
client = MongoClient('mongodb://act4dgem.cse.iitd.ac.in',serverSelectionTimeoutMS=500000000)
source_collection = client['media-db']['JK_location_articles']

In [2]:
# Forming the location dictionary : 'locations'.
file = open('locations.csv')
locations = file.read().split('\n')[:-1]
file.close()
locations[0] = locations[0].split(',')
for i in range(1,len(locations)):
    temp = locations[i].split(',')
    locations[i] = [temp[1],int(temp[0])]
locations = dict(locations[1:])

In [12]:
# Function to clear the article text.
def filter_data(data):
    # specific
    data = re.sub(r"won't", "will not", data)
    data = re.sub(r"can\'t", "can not", data)
    # general
    data = re.sub(r"n\'t", " not", data)
    data = re.sub(r"\'re", " are", data)
    data = re.sub(r"\'s", " is", data)
    data = re.sub(r"\'d", " would", data)
    data = re.sub(r"\'ll", " will", data)
    data = re.sub(r"\'t", " not", data)
    data = re.sub(r"\'ve", " have", data)
    data = re.sub(r"\'m", " am", data)
    data = re.sub("\S*\d\S*", "", data)
    data = re.sub('[^A-Za-z0-9]+', ' ', data)
    return data.lower()


In [17]:
# Collect article data.
file1 = open('file1.csv','w')        # article_id, locations
file2 = open('file2.csv','w')        # article_id, date, month, year, sentiment_score
file3 = open('file3.csv','w')        # article_id, text

articles = source_collection.find({}).limit(100)
processed = 0
for article in articles:
    flag = False
    for entity in article['entities']:
        if entity['type']=='ProvinceOrState' or entity['type']=='City':
            if entity['name'].lower() in locations:
                flag=True
                file1.write(str(article['_id'])+','+str(locations[entity['name'].lower().strip()])+'\n')
    if flag:
        date = str(article['publishedDate'])
        date = date[:4]+','+date[5:7]+','+date[8:]
        file2.write(str(article['_id'])+','+date+','+str(article['sentiStrength'])+'\n')
        file3.write(str(article['_id'])+','+filter_data(str(article['articleTitle']))+','+filter_data(str(article['text']))+'\n')
    processed+=1
    print(processed)

file1.close()
file2.close()
file3.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
